In [5]:
import importlib
import sys
from tqdm import tqdm
import  os
import cv2
import glob
from datetime import datetime 
# Add the directories to the sys.path
dir_path = "/usera/jd2052/Documents/anubis/" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Visual_tools as VTools
import Reconstruction_tools as RTools
import mplhep as hep
import pickle
import Timing_tools as TTools
import overview
import rawFileReader
hep.style.use([hep.style.ATLAS])

# Specify the directory
#data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

#file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list)) # insert your file
#print(file_path)

In [ ]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(VTools)

file_name = 'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240815_1759.raw'
storage_name = "weird_data.pkl"

start = "2024-08-15 17:24:22"
end = "2024-08-15 17:24:23"


chunks, times, fReader = overview.get_chunks(file_name, start= start, end = end)

In [ ]:
with open(storage_name, "wb") as outp:
    pickle.dump([chunks, times], outp)

In [7]:
importlib.reload(overview)
import matplotlib.pyplot as plt

storage_name = "data/chunks_hv4.pkl"
with open(storage_name, "rb") as inp:
    total_chunks = pickle.load(inp)
print("Loaded chunks")

Loaded chunks


In [14]:
len(total_chunks) #chunks

179

In [8]:
#Create the images
total_chunks = total_chunks[-1][-1] #chunks

max_process_event_chunk = 1

processedEvents = 0
interval = 50
reconstructor = proAnubis_Analysis_Tools.Reconstructor(total_chunks[0], processedEvents)
with tqdm(total=max_process_event_chunk*interval, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        for evt_num, evt in enumerate(total_chunks[processedEvents]):
            #[print(hit) for hit in VTools.all_hits_event(evt)]
            maps = VTools.hitHeatMap(evt)
            for rpc in range(6):
                plt.imshow(maps[rpc], interpolation='nearest')
                plt.title(f'Event {evt_num} RPC {rpc}')
                plt.savefig(f'video/images_video/{processedEvents*100+10*evt_num+rpc}.png')
            pbar.update(1)

Processing Events: 101Events [29:00, 17.23s/Events]                             


KeyboardInterrupt: 

In [9]:
#Compose the video
image_folder = 'video/images_video'
images = [img for img in os.listdir(image_folder)[1::6] if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape


video_name = f'video/one_chunk.mp4'
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 8, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()